In [1]:
#all imports
import requests
import pandas as pd
from pathlib import Path
import numpy as np
import os

In [2]:
idx = pd.IndexSlice  #magically make MultiIndex slicing better

In [3]:
#create and reduce list of countries
#countries=pd.read_excel('raw_data/UNSD — Methodology.xlsx', sheetname='Sheet1')
#countries=countries.dropna(axis=0, subset=['ISO-alpha3 Code'])

In [4]:
#load original data
trade_data=pd.read_csv("raw_data/DataJobID-1257172_1257172_TestQuery.csv" , encoding = "ISO-8859-1")

In [5]:
#create data structures for building multi index dataframe
years = list(range(1995,2016))
countries=list(trade_data['ReporterName'].unique())

In [6]:
#TODO remove these
# countries_to_remove=['Belgium-Luxembourg', 'European Union', 'Other Asia, nes',' World']
# for country in countries_to_remove:
#     trade_data_export = trade_data_export[trade_data_export.Reporter != country]

In [7]:
def build_multi_index_df(years, countries):
    """
    Function which builds multi index dataframe suitable for this analysis.
    MultiIndex array is three-dimensional: year*country*country
    """ 

    #build MultiIndex array
    rows_array = []
    for year in years:
        for country in countries:
            rows_array.append([year,country])
        
    multi_index = pd.MultiIndex.from_tuples(rows_array, names=['year', 'country'])
    data = pd.DataFrame(columns=countries, index=multi_index)
    return data



In [8]:
#create dataframe structure
data = build_multi_index_df(years,countries)

In [9]:
#fill dataframe with values from data
#Caution, takes roughly 5 minutes!
for index, row in trade_data.iterrows():
    for year in years:
        year_key=str(year)+" in 1000 USD "
        data.loc[year][row['ReporterName']][row['PartnerName']]=row[year_key]


In [10]:
#fill NaNs
data_filled=data.fillna(0)

In [12]:
#write to TSV
data_filled.to_csv('trade_data.tsv', sep='\t')

In [13]:
#read from TSV
imported_data = pd.read_table('trade_data.tsv', index_col=[0,1])

In [18]:
#compare whether identical
all(imported_data == data_filled)

True